# Garmin Connect data importer

This script moves data from https://connect.garmin.com/modern/activities *.csv export to DuckDB database for further use

In [ ]:
import duckdb
import pandas as pd
from data.paths import DB_PATH, CSV_PATH, specific_csv

df = pd.read_csv(specific_csv, sep=",", decimal=",", na_values="--", thousands=".")

Makes csv into dataframe using pandas, while in this step it renames the original Czech column names with English ones to ensure correct encoding and limiting spaces.

In [ ]:
# mapping from Garmin Czech CSV headers → internal standardized names
mapping = {
    "Typ aktivity": "activity_type",
    "Datum": "date",
    "Oblíbený": "favourite",
    "Název": "name",
    "Vzdálenost": "distance_km",
    "Kalorie (kcal)": "calories_kcal",
    "Čas": "duration",
    "Průměrný ST": "avg_hr",
    "Maximální ST": "max_hr",
    "Aerobní TE": "aerobic_te",
    "Průměrná kadence": "avg_cadence",
    "Maximální tempo": "max_cadence",
    "Průměrné tempo": "avg_pace",
    "Nejlepší tempo": "best_pace",
    "Celkový výstup": "total_ascent_m",
    "Celkový sestup": "total_descent_m",
    "Průměrná délka kroku": "avg_stride_len_m",
    "Training Stress Score®": "training_stress_score",
    "Kroky": "steps",
    "Minimální teplota": "min_temp_c",
    "Dekomprese": "decompression",
    "Čas nejlepšího okruhu": "best_lap_time",
    "Počet okruhů": "lap_count",
    "Maximální teplota": "max_temp_c",
    "Čas pohybu": "moving_time",
    "Uplynulý čas": "elapsed_time",
    "Minimální nadmořská výška": "min_alt_m",
    "Maximální nadmořská výška": "max_alt_m"
}

all_cols = list(mapping.values())


def standardize_garmin_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize Garmin CSV columns to a fixed schema even if some are missing or reordered."""
    # Rename known columns
    df.rename(columns=mapping, inplace=True)

    # Add any missing columns as None
    for col in all_cols:
        if col not in df.columns:
            df[col] = pd.NA

    # Keep only known columns, ignore extras
    df = df[[col for col in all_cols if col in df.columns]]

    # Optional: warn about unexpected/unmapped columns
    unknown_cols = [c for c in df.columns if c not in all_cols]
    if unknown_cols:
        print("⚠️ Unmapped columns found in CSV:", unknown_cols)

    return df

df = standardize_garmin_columns(df)
#print(df.head())


### Database preparation

* connects to database
* truncates tables to ensure I do not have data duplicates

* for now truncates also the final destination table since there is very little data and there is currently no "no-duplicates" code implemented

In [ ]:
conn = duckdb.connect(DB_PATH)

truncate = True

if truncate:
    try:
        conn.execute("""TRUNCATE TABLE stg_activity;""")
        conn.execute("""TRUNCATE TABLE tbl_activity;""")
    except Exception as e:
        print(f"⚠️  Error while truncating tables: {e}")
else:
    print("Truncate skipped")

This part is basically just for testing, when I need to delete tables on regular basis.

In [ ]:
drop = False

if drop:
    try:
        conn.execute("""DROP TABLE stg_activity;""")
        conn.execute("""DROP TABLE tbl_activity;""")
        conn.execute("""DROP VIEW vw_activity;""")
    except Exception as e:
        print(f"Error while dropping table: {e}")
else:
    print("ℹ️ Table drop skipped.")

First I need to create both tables in the database to ensure I can work with them. It will try to create them only if they do not exist.

In [ ]:

conn.execute("""
CREATE TABLE IF NOT EXISTS stg_activity (
    activity_type         VARCHAR,      -- Typ aktivity
    date                  VARCHAR,      -- Datum
    favourite             BOOLEAN,      -- Oblíbený
    name                  VARCHAR,      -- Název
    distance_km           DOUBLE,       -- Vzdálenost (km)
    calories_kcal         INTEGER,      -- Kalorie (kcal)
    duration              INTERVAL,     -- Čas (celkový)
    avg_hr                SMALLINT,     -- Průměrný ST (tep)
    max_hr                SMALLINT,     -- Maximální ST (tep)
    aerobic_te            DOUBLE,       -- Aerobní TE (training effect)
    avg_cadence           SMALLINT,     -- Průměrná kadence
    max_cadence           INTEGER,      -- Maximální tempo (min/km or m/s)
    avg_pace              VARCHAR,      -- Průměrné tempo
    best_pace             VARCHAR,      -- Nejlepší tempo
    total_ascent_m        DOUBLE,       -- Výstup
    total_descent_m       DOUBLE,       -- Celkový sestup
    avg_stride_len_m      DOUBLE,       -- Průměrná délka kroku (m)
    training_stress_score DOUBLE,       -- Training Stress Score®
    steps                 INTEGER,      -- Kroky
    min_temp_c            DOUBLE,       -- Minimální teplota
    decompression         VARCHAR,      -- Dekomprese (text/flag)
    best_lap_time         INTERVAL,     -- Čas nejlepšího okruhu
    lap_count             SMALLINT,     -- Počet okruhů
    max_temp_c            DOUBLE,       -- Maximální teplota
    moving_time           INTERVAL,     -- Čas pohybu
    elapsed_time          INTERVAL,     -- Uplynulý čas
    min_alt_m             DOUBLE,       -- Minimální nadmořská výška
    max_alt_m             DOUBLE        -- Maximální nadmořská výška
);            
""")

In [ ]:
conn.execute("""
CREATE TABLE if not exists tbl_activity (
    activity_type       TEXT,
    activity_date       DATE,
    activity_time       TIME,
    favourite           BOOLEAN,
    activity_name       TEXT,
    distance_km         DOUBLE,
    calories_kcal       INTEGER,
    duration            INTERVAL,  
    avg_hr              SMALLINT,
    max_hr              SMALLINT,
    aerobic_te          DOUBLE,
    avg_cadence         SMALLINT,
    max_cadence         INTEGER,
    steps               INTEGER,
    avg_pace            INTERVAL,
    best_pace           INTERVAL,
    avg_speed           DOUBLE,
    best_speed          DOUBLE,
    total_ascent        DOUBLE,
    total_descent       DOUBLE,
    avg_stride_lenght   DOUBLE,
    training_stress     DOUBLE,
    min_temp_c          DOUBLE,
    max_temp_c          DOUBLE,
    moving_time         INTERVAL,
    elapsed_time        INTERVAL,
    best_time_lap       INTERVAL,
    lap_count           SMALLINT,
    min_alt_m           DOUBLE,
    max_alt_m           DOUBLE,
    decompression       TEXT
);
""")

### Database modification

This part actually tries to upload data to a database using the dataframe we have created early in the script.

In [ ]:
conn.register("df", df)
conn.execute("INSERT INTO stg_activity SELECT * FROM df")

I need to create a view as a midstep due to some calculations to not create a messy and hard to read code.

In [ ]:
conn.execute("""
    CREATE VIEW if not exists vw_activity AS
        SELECT 
        activity_type,
        CAST(SPLIT_PART(date, ' ', 1) AS DATE) AS activity_date,
        CAST(SPLIT_PART(date, ' ', 2) AS TIME) AS activity_time,
        favourite,
        NAME AS activity_name,
        CAST(distance_km AS DOUBLE) AS distance_km,
        CAST(calories_kcal AS INTEGER) AS calories_kcal,
        CAST(duration AS INTERVAL) AS duration,
        CAST(avg_hr AS SMALLINT) AS avg_hr,
        CAST(max_hr AS SMALLINT) AS max_hr,
        CAST(aerobic_te AS DOUBLE) AS aerobic_te,
        CAST(avg_cadence AS SMALLINT) AS avg_cadence,
        CAST(max_cadence AS INTEGER) AS max_cadence,
        CAST(steps AS INTEGER) AS steps,
        TRY_CAST(CONCAT('00:', avg_pace::TEXT) AS INTERVAL) AS avg_pace,
        TRY_CAST(CONCAT('00:', best_pace::TEXT) AS INTERVAL) AS best_pace,
        CASE 
            WHEN avg_pace NOT LIKE '%:%'                            -- min/km or km/h will be correctly moved to their columns and datatypes changed accordingly
            THEN CAST(REPLACE(avg_pace, ',', '.') AS DOUBLE)
            ELSE NULL 
        END AS avg_speed,
        CASE 
            WHEN avg_pace NOT LIKE '%:%'                            --  min/km or km/h will be correctly moved to their columns and datatypes changed accordingly
            THEN CAST(REPLACE(best_pace, ',', '.') AS DOUBLE)
            ELSE NULL 
        END AS best_speed,
        CAST(total_ascent_m AS DOUBLE) AS total_ascent,
        CAST(total_descent_m AS DOUBLE) AS total_descent,
        CAST(avg_stride_len_m AS DOUBLE) AS avg_stride_lenght,
        CAST(training_stress_score AS DOUBLE) AS training_stress,
        CAST(min_temp_c AS DOUBLE) AS min_temp_c,
        CAST(max_temp_c AS DOUBLE) AS max_temp_c,
        CAST(moving_time AS INTERVAL) AS moving_time,
        CAST(elapsed_time AS INTERVAL) AS elapsed_time,
        CAST(best_lap_time AS INTERVAL) AS best_time_lap,
        CAST(lap_count AS SMALLINT) AS lap_count,
        CAST(min_alt_m AS DOUBLE) AS min_alt_m,
        CAST(max_alt_m AS DOUBLE) AS max_alt_m,
        decompression
    FROM stg_activity;
""")

Cast and change datatypes since the first data dump used some columns as TEXT.

Those columns are DATETIME issue loosing the TIME part.

Second was about average and best pace, since those are for some activities as INTERVAL (min/km) and for some as DOUBLE (km/h), therefore split based on this character's ":" existence.

In [ ]:
# Insert into the final table
conn.execute("""
INSERT INTO tbl_activity
SELECT
    activity_type,
    activity_date,
    activity_time,
    favourite,
    activity_name,
    distance_km,
    calories_kcal,
    duration,
    avg_hr,
    max_hr,
    aerobic_te,
    avg_cadence,
    max_cadence,
    steps,
    avg_pace,
    best_pace,
    CASE 
		WHEN avg_speed IS NULL THEN 
			ROUND(
		        3600.0 / (EXTRACT(HOUR FROM avg_pace) * 3600 
                 + EXTRACT(MINUTE FROM avg_pace) * 60 
                 + EXTRACT(SECOND FROM avg_pace)),
             2)
		ELSE avg_speed
	END AS avg_speed,
	best_speed,
    total_ascent,
    total_descent,
    avg_stride_lenght,
    training_stress,
    min_temp_c,
    max_temp_c,
    moving_time,
    elapsed_time,
    best_time_lap,
    lap_count,
    min_alt_m,
    max_alt_m,
    decompression
FROM vw_activity;
""")

conn.close()